## Content-Based E-Commerce Product Recommendation System with Price-Aware Search

### Project Goal (What We Want to Achieve)

->The goal of this project is to recommend relevant e-commerce products based on:

->User search text (product type, brand, category)

->User price preference (exact price, price range, under budget)

->The system ensures that when a customer searches:

->“sneakers 10000”

->they receive top relevant products near ₹10,000, not random products across a wide price range.

## Dataset Explanation
### 🧾Dataset Used

A structured e-commerce dataset containing product-level information.

#### Features (Columns)

##### Feature Name	&   Description

-> product_name	        (Name of the product)

-> category	            (Product category (shoes, clothing, electronics, etc.))

-> brand	            (Brand name)

-> description	        (Detailed product description)

-> price	            (Product price (numeric))

-> rating (optional)	(Customer rating)

## Input Features (Model Input)

### Textual Features (used for similarity)

-> product_name

-> category

-> brand

-> description

### User Query

->Search text (e.g., sneakers, nike shoes)

->Price constraint (e.g., 10000, under 5000, 8000–12000)

### Output Features (Model Output)

##### Output    &      Description

-> product_name	      (Recommended product)

-> price	          (Product price)

-> score	          (Similarity score (cosine similarity))

### Project Workflow (Overall Flow)

User Query

   ↓

Extract Price Constraint

   ↓

Clean & Extract Keywords

   ↓

Keyword / Category Filtering

   ↓

Price Filtering

   ↓

TF-IDF Vectorization

   ↓

Cosine Similarity Scoring

   ↓
   
Top-N Recommendations


## Step-by-Step Project Implementation

### STEP 1: Import Required Libraries

In [2]:
import pandas as pd
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


### STEP 2: Dataset Loading

 -> Purpose

 Load the e-commerce dataset into memory.

 -> Code Concept

In [3]:
import pandas as pd
df = pd.read_csv("products.csv")

#Inbuilt Functions Used

#pd.read_csv() → Reads CSV file into DataFrame

###  STEP 3: Data Cleaning & Preprocessing

##### Purpose

->Ensure data quality before training.

##### Operations Performed

->Remove missing values

->Convert price to numeric

->Reset index


##### Inbuilt Functions Used

Function &	Purpose

dropna()	Remove missing values

astype()	Convert data type

reset_index()	Reset DataFrame index

In [4]:
df = df.dropna(subset=['product_name', 'price'])

df['price'] = df['price'].astype(int)

df.reset_index(drop=True, inplace=True)


### STEP 4: Feature Engineering (Text Combination)

##### Purpose

->Create a single text feature for TF-IDF processing.

##### Combined Text Column

In [5]:
df['combined_text'] = (
    df['product_name'].astype(str) + ' ' +
    df['category'].astype(str) + ' ' +
    df['brand'].astype(str) + ' ' +
    df['description'].astype(str)
)

#Why?

#TF-IDF works on one text column only, so we combine all text features.
# - TF‑IDF (Term Frequency–Inverse Document Frequency) is a technique 
# that converts text into vectors or numerical features for machine learning.

### STEP 5: TF-IDF Vectorization

##### Purpose
-> Convert text into numerical vectors.

##### Algorithm Used

-> TF-IDF (Term Frequency–Inverse Document Frequency)

-> Inbuilt Classes & Methods

##### Function/Class & Purpose---

-> TfidfVectorizer()	Text vectorization

-> fit_transform()	Learn vocabulary + transform text


##### Why TF-IDF?

-> Penalizes common words

-> Highlights important product terms

-> Efficient for content-based filtering

In [6]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000
)

product_vectors = tfidf.fit_transform(df['combined_text'])


### STEP 6: Extract Price from User Query

##### Purpose

-> Handle price logically instead of treating it as text.

##### Supported Formats

-> Exact price → 10000

-> Range → 8000-12000

-> Under budget → under 5000

##### Inbuilt Functions Used

Function &	   Purpose

-> re.search()	   Pattern matching

-> lower()	       Normalize input

In [8]:
#Converts user input to lowercase for consistency
def extract_price(query):
    query = query.lower()
    
    #Detects price range like 8000-12000
    range_match = re.search(r'(\d+)\s*-\s*(\d+)', query)
    
    #Extracts minimum and maximum price from range
    if range_match:
        return int(range_match.group(1)), int(range_match.group(2))
    
    #Detects queries like under 5000
    under_match = re.search(r'(under|below)\s*(\d+)', query)

    #Sets minimum price as 0 and maximum as given value
    if under_match:
        return 0, int(under_match.group(2))
    
    #Detects queries like 10000
    exact_match = re.search(r'\b(\d{3,6})\b', query)

    #Adds ±500 buffer for realistic matching
    if exact_match:
        price = int(exact_match.group(1))
        return price - 500, price + 500
    
    #If no price is mentioned, no filtering is applied
    return None, None


### STEP 7: Clean User Query Text

##### Purpose

-> Remove numeric values from query before vectorization.

Example 

"sneakers 10000" → "sneakers"


##### Inbuilt Functions Used

Function & Purpose


-> re.sub()	Remove numbers

-> strip()	Remove extra spaces

In [9]:
def clean_query(query):
    return re.sub(r'\d+', '', query).strip()


Explanation

re.sub(r'\d+', '', query)
→ Removes numbers from text

.strip()
→ Removes extra spaces

Example:

"sneakers 10000" → "sneakers"

### STEP 8: Extract Category / Keyword from Query

We already cleaned numbers, but we need strong keyword matching.

In [10]:
def extract_keywords(query):
    keywords = query.lower()
    keywords = re.sub(r'\d+', '', keywords)
    return keywords.strip()


### STEP 9: Similarity Calculation

##### Purpose

-> Measure relevance between user query and products.

##### Algorithm Used

-> Cosine Similarity

##### Inbuilt Functions Used

Function & Purpose---

-> cosine_similarity()	Compute similarity score

-> flatten()	Convert to 1-D array

In [11]:
def calculate_similarity(text_query, filtered_df, tfidf):
    """
    Calculates cosine similarity between user query and filtered products
    """

    # Convert product text to vectors
    product_vectors = tfidf.transform(filtered_df['combined_text'])

    # Convert query text to vector
    query_vector = tfidf.transform([text_query])

    # Calculate cosine similarity
    similarity_scores = cosine_similarity(
        query_vector, product_vectors
    ).flatten()

    return similarity_scores


### STEP 10: Ranking & Recommendation

##### Purpose

-> Return top-N most relevant products.

##### Inbuilt Functions Used

Function &	Purpose

-> sort_values()	Rank products

-> head()	Select top results

In [12]:
def rank_and_recommend(filtered_df, similarity_scores, top_n=10):
    """
    Ranks products based on similarity score and returns top-N results
    """

    # Add similarity score to dataframe
    filtered_df = filtered_df.copy()
    filtered_df['score'] = similarity_scores

    # Sort by relevance
    results = filtered_df.sort_values(
        by='score', ascending=False
    ).head(top_n)

    return results[['product_name', 'price', 'score']]


### STEP 11: Testing the System 

In [13]:
#FINAL MAIN FUNCTION (CONNECTS EVERYTHING) 

def recommend_products(user_query, top_n=10):

    # 1️ Extract price range
    min_price, max_price = extract_price(user_query)

    # 2️ Extract clean keywords
    keywords = extract_keywords(user_query)

    # 3️Keyword-based filtering (CRITICAL)
    keyword_mask = (
        df['combined_text']
        .str.lower()
        .str.contains('|'.join(keywords.split()), regex=True)
    )

    filtered_df = df[keyword_mask]

    # 4️ Apply price filter
    if min_price is not None and max_price is not None:
        filtered_df = filtered_df[
            (filtered_df['price'] >= min_price) &
            (filtered_df['price'] <= max_price)
        ]

    if filtered_df.empty:
        return "No relevant products found"

    # 5️ TF-IDF similarity
    filtered_vectors = tfidf.transform(filtered_df['combined_text'])
    query_vector = tfidf.transform([keywords])

    similarity_scores = cosine_similarity(
        query_vector, filtered_vectors
    ).flatten()

    filtered_df = filtered_df.copy()
    filtered_df['score'] = similarity_scores

    # 6️ Sort and return top N
    result = filtered_df.sort_values(
        by='score', ascending=False
    ).head(top_n)

    return result[['product_name', 'price', 'score']]


In [15]:
recommend_products("laptops 10000-25000")

,product_name,price,score
102,Laptop Model 7,10531,0.343849
97,Laptop Model 2,17521,0.326990
98,Laptop Model 3,18996,0.320630
40,Men's Casual T-Shirt Variant 1,14589,0.000000
54,Men's Casual T-Shirt Variant 15,25000,0.000000
52,Men's Casual T-Shirt Variant 13,22278,0.000000
51,Men's Casual T-Shirt Variant 12,14120,0.000000
56,Girls Ethnic Dress Variant 2,20216,0.000000
66,Girls Ethnic Dress Variant 12,18408,0.000000
64,Girls Ethnic Dress Variant 10,17594,0.000000


In [16]:
recommend_products("sneakers shoes under 5000")

,product_name,price,score
7,Asian Men's Casual Sneaker Variant 8,3930,0.369785
3,Asian Men's Casual Sneaker Variant 4,1815,0.366039
2,Asian Men's Casual Sneaker Variant 3,4238,0.366039
0,Asian Men's Casual Sneaker Variant 1,2580,0.363579
6,Asian Men's Casual Sneaker Variant 7,3271,0.363579
1,Asian Men's Casual Sneaker Variant 2,3719,0.362131
4,Asian Men's Casual Sneaker Variant 5,1755,0.360489
10,Asian Men's Casual Sneaker Variant 11,2100,0.344556
12,Asian Men's Casual Sneaker Variant 13,3032,0.342244
18,Asian Men's Casual Sneaker Variant 19,4931,0.332651


In [17]:
recommend_products("nike shoes 8000-12000")

'No relevant products found'

Output

✔️ Relevant products
✔️ Correct price range
✔️ Ranked by similarity

### STEP 12: Model Persistence Using Pickle

In [31]:
import pickle

pickle.dump(tfidf, open("tfidf.pkl", "wb"))
pickle.dump(df, open("products.pkl", "wb"))


##### What is a Pickle File?

-> Pickle is a Python module used to save (serialize) Python objects into a file so they can be reused later without retraining.

##### In simple words:

-> Pickle converts a Python object → byte stream → file (.pkl)
And later converts the file → back to the original object.